# MCP와 전통적인 접근 방식 비교

## 전통적인 함수 호출 접근 방식

먼저, 일반적인 함수 호출 방식을 살펴본다. 이 방식은 필요한 함수를 직접 정의하고, OpenAI 같은 언어 모델(LLM)에 이 함수의 명세를 알려주어 필요할 때 호출하도록 하는 구조다.

### 1. 도구(함수) 정의

먼저, LLM이 사용할 간단한 `add` 함수를 `tools.py` 파일에 정의한다.

In [1]:
%%writefile tools.py

def add(a: int, b: int) -> int:
    """두 숫자를 더한다"""
    return a + b

Writing tools.py


### 2. 함수 호출 실행

다음 코드는 위에서 정의한 `add` 함수를 LLM에 도구로 제공하고, 사용자 요청에 따라 함수를 실행하는 전체 과정을 보여준다. 이 예제는 MCP가 함수를 호출하는 새로운 방법을 제공한다는 것을 보여주기 위한 간단한 예시다.

**실행 과정:**
1. **도구 명세 정의**: `tools` 변수에 LLM이 이해할 수 있는 형식으로 `add` 함수의 이름, 설명, 파라미터를 정의한다.
2. **LLM 호출**: 사용자의 메시지("25 + 17 계산해줘")와 도구 명세를 함께 LLM에 전달한다.
3. **도구 호출 처리**: LLM이 함수를 호출해야 한다고 판단하면, `tool_calls` 정보를 응답에 담아 보낸다.
4. **직접 실행**: 코드에서 `add` 함수를 직접 실행하여 결과를 얻는다.
5. **결과 전달**: 실행 결과를 다시 LLM에 전달하여 최종 응답을 생성하게 한다.

In [ ]:
# 파일명: function-calling.py

import json
import openai
from dotenv import load_dotenv
from tools import add

# .env 파일에서 환경변수(OPENAI_API_KEY)를 로드한다
load_dotenv("../.env")

# 모델에 제공할 도구를 정의한다
tools = [
    {
        "type": "function",
        "function": {
            "name": "add",
            "description": "두 숫자를 더한다",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {"type": "integer", "description": "첫 번째 숫자"},
                    "b": {"type": "integer", "description": "두 번째 숫자"},
                },
                "required": ["a", "b"],
            },
        },
    }
]


# LLM을 호출한다
response = openai.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": "25 + 17을 계산해줘"}],
    tools=tools,
)

# 도구 호출을 처리한다
if response.choices[0].message.tool_calls:
    tool_call = response.choices[0].message.tool_calls[0]
    tool_name = tool_call.function.name
    tool_args = json.loads(tool_call.function.arguments)

    # 함수를 직접 실행한다
    result = add(**tool_args)

    # 결과를 다시 모델에 보낸다
    final_response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": "25 + 17을 계산해줘"},
            response.choices[0].message,
            {"role": "tool", "tool_call_id": tool_call.id, "content": str(result)},
        ],
    )
    print(final_response.choices[0].message.content)

## 비교하기

이렇게 작은 규모에서는 전통적인 접근 방식이 더 간단하다. 하지만 다음과 같은 상황에서는 MCP 접근 방식과의 주요 차이점이 명확해진다:

1. **규모 증가**: 수십 개의 도구가 있는 경우, MCP 접근 방식이 더 나은 구성을 제공한다.
2. **재사용성**: MCP 서버는 여러 클라이언트와 애플리케이션에서 사용될 수 있다.
3. **분산 환경**: MCP는 원격 작업을 위한 표준 메커니즘을 제공한다.

## MCP와 전통적인 접근 방식 중 언제 사용해야 하는가?

### MCP를 고려해야 할 때:

- 여러 애플리케이션에서 도구 구현을 공유해야 할 때
- 서로 다른 컴퓨터에 있는 구성 요소로 분산 시스템을 구축할 때
- 생태계에 이미 존재하는 MCP 서버를 활용하고 싶을 때
- 표준화가 사용자에게 이점을 제공하는 제품을 구축할 때

### 전통적인 접근 방식이 더 나을 때:

- 더 간단하고 독립적인 애플리케이션을 만들 때
- 성능이 매우 중요할 때 (직접 함수 호출이 오버헤드가 적다)
- 개발 초기 단계로, 표준화보다 빠른 반복이 더 중요할 때